# Installation

*important you need to set verbose=True on Llamacpp args in order to make this workging on collab.*

In [ ]:
!df -h /content
!cat /proc/meminfo | grep MemTotal
!nvidia-smi

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
## run once restart vm, run a second time

!apt install -y build-essential
!pip install -q condacolab
import condacolab
condacolab.install()

!conda install -y cmake
!conda install -y cuda-toolkit cuda-nvcc

In [ ]:
import condacolab
condacolab.check()

In [ ]:
!pip install  kaleido python-multipart cohere


In [ ]:
%env HF_HUB_ENABLE_HF_TRANSFER=1

!mkdir -p logical_model
!rm -Rf augmentool
!pip install --upgrade pip
!pip install kaleido python-multipart cohere
!pip install -Uqq protobuf sentencepiece transformers matplotlib nltk aphrodite-engine hf-transfer ipdb huggingface-hub
!pip cache purge

!git clone  --branch master https://github.com/e-p-armstrong/augmentool.git
!mv -f augmentool/* .

!huggingface-cli download TheBloke/FlatOrcamaid-13B-v0.2-GGUF flatorcamaid-13b-v0.2.Q6_K.gguf --local-dir logical_model --local-dir-use-symlinks False
!huggingface-cli download TheBloke/Airoboros-L2-70B-3.1.2-GGUF airoboros-l2-70b-3.1.2.Q4_K_M.gguf --local-dir logical_model --local-dir-use-symlinks False


import ipdb

# Personnal config

In [8]:
##############################################################
##         YOUR PERSONNAL COLLAB CONFIG
##############################################################

LOGICAL_MODEL = "./logical_model/flatorcamaid-13b-v0.2.Q6_K.gguf"

LARGE_LOGICAL_MODEL = "./logical_model/airoboros-l2-70b-3.1.2.Q4_K_M.gguf"

ASSISTANT_MODE = True

DOUBLE_CHECK_COUNTER = 3

USE_SUBSET = True

REARRANGEMENTS_TO_TAKE = 3

source_texts = [
    "Simple Sabotage, by the Office of Strategic Services, published 1944.txt"
]

TEXT_MANUALLY_CLEANED=False


# Welcome to Augmentoolkit

This notebook is where you generate all your data.

Augmentoolkit is meant to allow instruct-tuned models to learn from books, even using themselves to generate new data through a sort-of bootstrapping method. It is meant to stop model creators from having to work as data annotators, and not actual model trainers. It is meant to allow anyone to make their own high-quality dataset with thousands of entries.

## Quickstart:

- Get this notebook and the other repo code onto a machine with the power to run Airoboros-l2-70b-3.1.2.Q4_K_M
- Put the model into ./logical_models (relative to this notebook).
- Run all the cells below and watch as the notebook generates questions, answers, and conversations based on Principles of Chemistry, Simple Sabotage, and Introduction to Philosophy.

If you want to add your own texts, follow the instructions in list item #1 above.

### Note: this notebook makes roughly 1/3 characters generated to be **mildly NSFW** by default. You will need to modify the character personality code in `./generation_functions/special_instructions.py` or use "Assistant mode" if you want something cleaner.

## Customization:
### Here are some ways you can adapt this notebook to your use case, along with a brief description of how to do so, arranged in increasing order of difficulty (this information is also available in the README):
1. ***Change the source texts used to generate training data.*** You can do this in the cell right below this one. **IMPORTANT** the filenames of these should be formatted in a specific way, since the filenames are used as part of the prompts and in at least one regex. You need to have them be like: `[textname], by authorname`. You can also include the publication date after the author name if you want, but note that this will tend to bias most of the characters to live in the era of the textbook, which may or may not be what you want.

2. ***Change the personalities of the characters generated.*** Currently, when generating characters for the multiturn conversation step, three randomly-selected traits are appended to the "special instructions" set of the prompt to constrain what kind of character is generated by the model. Depending on what kind of model you want to make, or even just if your preferences vary, then you will probably want to modify this a bit. You can do so in `./generation_functions/special_instructions.py`. A more in-depth description of the trait-axis system that I (over)thought up is available in the comments of that file.

3. ***Change the constants.*** There are a few constant values in this notebook, and in `./generation_functions/constant_values.py`. These constants are tested, but if your use case requires special settings (e.g., you want to make conversations from more permutations of existing questions; or you think the character counts for the "duplicate question/answer" validation functions are too restrictive) then feel free to change the related setting. The most intuitive and least-likely-to-break-anything settings to change are rearrangements_to_take and double_check_counter. Beyond that... you'll need to figure out what the function does before changing it if you expect it to run.

4. ***Assistant Mode*** Technically this could be considered part of 3), but it's different enough that I feel it warrants separate explanation. By default, the notebook is configured to produce RP-style data; "Assistant mode" is something you can toggle in the settings cell immediately below this one, which skips character and scenario generation and answers every question in a chat between a user and a helpful AI assistant (with no personality). In the limited testing I have done with this, **it seems that assistant mode is simple enough to work with 13b models** such as Flatorcamaid by Ikari. So if your compute or time are very limited, or you are using this for a more professional use case, feel free to turn this on.

5. ***Change the model.*** This is as simple as switching the LOGICAL_MODEL parameter out for another one, but your mileage may vary, significantly. You will also have to adjust RoPE scaling for non-llama 2 models -- e.g., if you're using Mixtral, don't leave `rope_freq_scale=0.33`, which 3xes the context (you do not need 96k context, only 12k).

6. ***Change the examples.*** If you change the examples you can completely overhaul what this notebook does, but this requires a lot of prompting skill and possibly huge amounts of time to get it working again (source: most of my last three months were spent prompting, and most of this prompting was spent on the examples). Unless you want to convert this notebook from question-and-answer generation to some completely other task, I'd recommend changing only the conversation generation prompts -- they're a bit less finnicky, and if you just want to change the kind of characters generated (maybe you want a different writing style) that's where you'd find the differences.



In [ ]:
# NOTE NOTEBOOK SETTINGS AND CONSTANTS (some script file constants are in generation_functions/constants.py)

# Put your desired quant of your desired model in the relevant directories


# "airoboros-l2-70b-3.1.2.Q4_K_M.gguf" <- recommended for the large logical model
# "flatorcamaid-13b-v0.2.Q8_0.gguf" <- recommended for the normal logical model
# A6000s on Vast.ai are a good choice for running this notebook

LOGICAL_MODEL = "./logical_model/flatorcamaid-13b-v0.2.Q8_0.gguf"  # model used for decision-making and base question generation (should be "smart")
LARGE_LOGICAL_MODEL = "./logical_model/airoboros-l2-70b-3.1.2.Q4_K_M.gguf"
ASSISTANT_MODE = False  # change to true if you want all conversations to be with an "AI language model" and not characters. Useful for more professional use cases.
DOUBLE_CHECK_COUNTER = 3  # Set to 1 to check outputs only once; set to 2 to check twice; set to 3 to check thrice, etc. Set to 0 to break everything in vet_question_loop() and elsewhere. Set to -1 and cause the universe to implode?

REARRANGEMENTS_TO_TAKE = 3  # How many of the possible permutations of tuples in a group to take and make multiturn convs out of. Adjust higher to get more data out of less text, but it might be a bit repetitive. NOTE your eval loss will be basically worthless if you aren't careful with how you shuffle your dataset when you're about to train.

TEXT_MANUALLY_CLEANED = False  # If you've manually cut out all the parts of the text not worthy for questions, you can skip the first LLM step. NOTE I might actually recommend doing this if your source text is small, given how permissive the filtering prompt is; it really only disallows metadata.

source_texts = [
    "Simple Sabotage, by the Office of Strategic Services, published 1944.txt",
    "Principles of Chemistry, by Demitry Mendeleev, published 1897.txt",
]

# Below: Defines and imports functions that you will probably use no matter what cells in the notebook you choose to run:

### Also: The below will absolutely flood your terminal, due to heavy gbnf grammar use. I don't think this can be configured. Be prepared to scroll to reach the next cell.

In [ ]:
import os
import uuid

multi_turn_convs_info_dir = "./multi_turn_convs_info"  # we generate all the information fed to the multiturn prompt, and generate the actual multiturn prompt, separately; since every step but the last is capable of being done by a 13b


def make_id():
    return str(uuid.uuid4())


def write_output_to_file(output, directory, uuid):
    # Ensure directory exists
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Define the file path using the directory and UUID
    file_path = os.path.join(directory, f"{uuid}.txt")

    # Write the output to the file
    with open(file_path, "w") as file:
        file.write(output)

    print(f"Output written to {file_path}")

    # This is in no way best practices, but all my prompts being searchable and separate files is a good way to make my life easier.


import pkgutil
import importlib
import generation_functions  # This is the package directory
import sys

sys.path.append("./generation_functions")

# First, import all modules so they can be reloaded
for _, module_name, _ in pkgutil.iter_modules(
    generation_functions.__path__, generation_functions.__name__ + "."
):
    importlib.import_module(module_name)

# Now, reload each module and import all callable attributes
for _, module_name, _ in pkgutil.iter_modules(
    generation_functions.__path__, generation_functions.__name__ + "."
):
    # Reload the module
    module = importlib.reload(sys.modules[module_name])
    # Iterate through each attribute in the reloaded module
    for attribute_name in dir(module):
        # Retrieve the attribute
        attribute = getattr(module, attribute_name)
        if callable(attribute):
            # If it's callable, it's a function or class, so you set it in the globals dictionary
            globals()[attribute_name] = attribute


def make_multiturn_conversation(info, logic_llm):
    conv, conv_output = multi_turn_conversation(
        info[0], info[1], info[2], info[3], logic_llm, assistant_mode=ASSISTANT_MODE
    )  # based on what was originally: multi_turn_conversation(qa_tuples, character, scenario, scenario_plan, logic_llm)
    write_output_to_file(conv_output, "./multiturn_conversation_generations", info[4])

    return conv


def ensure_multiple_answers_are_same(
    info, conv, scenario
):  # why is this a whole separate function? Once upon a time, LLMs were used in validation here, too. But programmatic validation SEEMS to catch the common problems. This is here so that I can add it back in if I have to.
    """Loop to ensure that the answer is consistent in the conversation and in the tuple."""
    retries = 0
    c = conv
    while retries < 2:  # try twice, since multiturn is an expensive operation
        if call_all_processors(c[0], info[0]):  # if programmatic validation passes
            return c

        retries += 1
        if retries >= 2:
            return None
        # If we're here, majority of relevance checks failed
        print("----------------\n\n\n\nRETRYING!!!!\n\n\n\n----------------")
        # Broken info is 1) rare and 2) handled by the retry limit. We don't want to waste compute on regenerating info as they take time.
        retry = make_multiturn_conversation(info, logic_llm)
        if retry is not None:  # Note: retry CANNOT actually be None
            c = retry
        else:
            # If we failed to generate a retry, don't waste compute
            return None

    return None

In [ ]:
from transformers import AutoTokenizer
import re
from tqdm import tqdm
import nltk

nltk.download("punkt")
from nltk.tokenize import sent_tokenize

tokenizer = AutoTokenizer.from_pretrained(
    "Gryphe/MythoMax-L2-13b"
)  # It doesn't matter what model goes here as long as it is sentencepiece


def sentence_chunking_algorithm(file_path, tokenizer, max_token_length=400):
    """
    This function takes a plaintext file and chunks it into sentences.

    :param file_path: Path to the plaintext file
    :param tokenizer: SentencePiece tokenizer
    :param max_token_length: The maximum token length for a chunk of sentences
    :return: List of sentence chunks with source text information
    """
    sentence_chunks_with_source = []
    current_chunk = []
    token_count = 0
    source_name = file_path.replace(".txt", "")

    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Remove Gutenberg header and footer
    content = re.sub(
        r"^.*?START OF (THIS|THE) PROJECT GUTENBERG EBOOK.*$\n",
        "",
        content,
        flags=re.MULTILINE,
    )
    content = re.sub(
        r"^.*?END OF (THIS|THE) PROJECT GUTENBERG EBOOK.*$\n",
        "",
        content,
        flags=re.MULTILINE,
    )

    sentences = sent_tokenize(content)

    for sentence in tqdm(sentences, desc=f"Processing {file_path}"):
        sentence_token_count = len(tokenizer.encode(sentence))

        if token_count + sentence_token_count <= max_token_length:
            current_chunk.append(sentence)
            token_count += sentence_token_count
        else:
            sentence_chunks_with_source.append((" ".join(current_chunk), source_name))
            current_chunk = [sentence]
            token_count = sentence_token_count

    # Add the last chunk if it exists
    if current_chunk:
        sentence_chunks_with_source.append((" ".join(current_chunk), source_name))

    return sentence_chunks_with_source


sentence_chunks = []
for source_text in source_texts:
    sentence_chunks += sentence_chunking_algorithm(source_text, tokenizer)


def fix_text(to_replace_arr, text):
    for startup in to_replace_arr:
        text = text.replace(startup[0], startup[1])
    return text


conversions = [("\n", " "), ("  ", " ")]

paragraphs_processed = [
    (fix_text(conversions, seq[0]), seq[1]) for seq in sentence_chunks
]

In [ ]:
len(paragraphs_processed)

In [ ]:
paragraphs_processed[5]

In [ ]:
# Load model into vram
from llama_cpp import Llama

logic_llm = Llama(
    model_path=LOGICAL_MODEL,
    n_gqa=8,
    offload_kqv=True,
    n_ctx=4000,
    rope_freq_scale=0.33,
    n_gpu_layers=50,
    verbose=True,
)  # load the logical LLM and offload everything

In [16]:
import json
import os
from tqdm import tqdm

# Create directory if it doesn't exist
output_dir = "./worthy_for_questions"
os.makedirs(output_dir, exist_ok=True)

# Determine which paragraphs are worthy of making questions from
if not TEXT_MANUALLY_CLEANED:
    judged_worthy_for_questions = []
    for idx, p in tqdm(enumerate(paragraphs_processed)):
        # for idx, p in tqdm(enumerate(paragraphs_processed[:10])):
        file_name = f"{idx}.json"
        file_path = os.path.join(output_dir, file_name)

        # Check if the judgement for this paragraph already exists
        if os.path.isfile(file_path):
            with open(file_path, "r") as file:
                data = json.load(file)
                print("LOADING: ", data)
            if isinstance(data, str):
                judged_worthy_for_questions.append((None, data[7:]))
            else:
                judged_worthy_for_questions.append(
                    (data["paragraph"], data["metadata"])
                )
        else:
            judgement = judge_paragraph(p, logic_llm)
            judged_worthy_for_questions.append(judgement)
            print("##################################################")
            print(judgement)
            print("##################################################")
            # Prepare the data to be written to the file
            if judgement is None:
               data_to_write = f"failed|{p}"

            elif judgement[0] is not None:
                # The paragraph passed the judgement
                data_to_write = {"paragraph": judgement[0], "metadata": judgement[1]}
            else:
                # The paragraph did not pass the judgement
                data_to_write = f"failed|{judgement[1]}"

            # Write the judgement to a unique file as JSON
            with open(file_path, "w") as file:
                json.dump(data_to_write, file)

            # Debug messages
            try:
                if judgement[0] is not None:
                    print(f"DEBUG model decided that index {idx} was suitable")
                else:
                    print(f"DEBUG model decided that index {idx} was not suitable")
            except:
                print(f"DEBUG max retries exceeded for index {idx}")

else:
    judged_worthy_for_questions = paragraphs_processed
    # No need to write to file since paragraph chunking is deterministic

0it [00:00, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =    1365.31 ms
llama_print_timings:      sample time =     429.73 ms /    17 runs   (   25.28 ms per token,    39.56 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1353.49 ms /    17 runs   (   79.62 ms per token,    12.56 tokens per second)
llama_print_timings:       total time =    1988.86 ms /    18 tokens
Llama.generate: prefix-match hit



Step 1. Identify Paragraph Content: This paragraph is from the introduct



llama_print_timings:        load time =    1365.31 ms
llama_print_timings:      sample time =     233.57 ms /    17 runs   (   13.74 ms per token,    72.78 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1340.45 ms /    17 runs   (   78.85 ms per token,    12.68 tokens per second)
llama_print_timings:       total time =    1726.38 ms /    18 tokens
Llama.generate: prefix-match hit



Step 1. Identify Paragraph Content: This paragraph is from "Simple Sab



llama_print_timings:        load time =    1365.31 ms
llama_print_timings:      sample time =     355.45 ms /    17 runs   (   20.91 ms per token,    47.83 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1372.18 ms /    17 runs   (   80.72 ms per token,    12.39 tokens per second)
llama_print_timings:       total time =    1947.08 ms /    18 tokens
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    1365.31 ms
llama_print_timings:      sample time =     150.48 ms /    18 runs   (    8.36 ms per token,   119.62 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1638.11 ms /    18 runs   (   91.01 ms per token,    10.99 tokens per second)
llama_print_timings:       total time =    1873.31 ms /    19 tokens
Llama.generate: prefix-match hit



Step 1. Identify Paragraph Content: This paragraph contains sabotage techniques target



llama_print_timings:        load time =    1365.31 ms
llama_print_timings:      sample time =     146.98 ms /    18 runs   (    8.17 ms per token,   122.47 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1624.97 ms /    18 runs   (   90.28 ms per token,    11.08 tokens per second)
llama_print_timings:       total time =    1857.61 ms /    19 tokens
17it [02:23,  8.43s/it]


Step 1. Identify Paragraph Content: This paragraph contains sabotage techniques target
##################################################
None
##################################################
DEBUG max retries exceeded for index 16


ValueError: Requested tokens (4002) exceed context window of 4000

In [ ]:
# Graphing code generated by GPT-4. May be suboptimal/ugly.
import matplotlib.pyplot as plt
from collections import Counter


def filter_and_graph(tuples):
    # Count the occurrences of None and non-None for each source text
    source_counts = Counter()
    for paragraph, source in tuples:
        if paragraph is None:
            source_counts[source] = source_counts.get(source, [0, 0])
            source_counts[source][0] += 1
        else:
            source_counts[source] = source_counts.get(source, [0, 0])
            source_counts[source][1] += 1

    # Prepare data for the graph
    labels = list(source_counts.keys())
    none_counts = [source_counts[source][0] for source in labels]
    non_none_counts = [source_counts[source][1] for source in labels]

    # Plotting the graph
    x = range(len(labels))
    plt.bar(x, none_counts, width=0.4, label="Not suitable", align="center")
    plt.bar(x, non_none_counts, width=0.4, label="Valid Paragraphs", align="edge")
    plt.xlabel("Source Text")
    plt.ylabel("Number of Paragraphs")
    plt.title("Paragraphs Suitable for Questions by Source Text")
    plt.xticks(x, labels, rotation="vertical")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Filter out tuples with None and return the new list
    filtered_list = [t for t in tuples if t[0] is not None]
    return filtered_list

In [ ]:
filtered_worthy_for_questions = filter_and_graph(judged_worthy_for_questions)

In [ ]:
print(filtered_worthy_for_questions[0])

In [ ]:
# Control flow helpers

# Change this value to change how many times the checks must pass consecutively for a thing to be accepted

import logging
from math import ceil


# Setup logging
# Except I actually don't use this because I switched to print() because jupyter is annoying with logging
logging.basicConfig(
    filename="data_generation.log",
    filemode="a",
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO,
)


def vet_answer_accuracy_loop(qa_tuple, total_retries, run_id):
    try:
        qtuple = qa_tuple
        print(
            f"\n\nStarting ACCURACY loop for question: {qtuple[0]}, context: {qtuple[2]}"
        )
        passed_checks = 0
        times_checked = 0
        dissenting_reasoning = ""
        while times_checked < DOUBLE_CHECK_COUNTER:
            print(
                f"\n\nACCURACY CALL CHECK ANSWER: {qtuple[0]}, context: {qtuple[2]}, retries: {total_retries}, dissenting reasoning: {dissenting_reasoning}"
            )
            judgement, answer_accuracy_output = check_answer(qtuple, logic_llm)
            write_output_to_file(
                answer_accuracy_output, "./check_answer_accuracy_generations", run_id
            )
            if not judgement[0]:  # if not accurate
                dissenting_reasoning = judgement[1]
            else:
                passed_checks += 1
            times_checked += 1
            if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                break
            failed_checks = times_checked - passed_checks
            if failed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                break

        if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):  # if question checks passed
            print(f"\n\ANSWER ACCURACY CHECKS PASSED retries: {total_retries}")
            return qtuple
        else:
            # Generate new question and restart the loop
            print(
                f"\n\nACCURACY CHECKS FAILED - SENDING BACK TO QUESTION LOOP retries: {total_retries}"
            )
            total_retries += 1
            qtuple, generate_new_q_output = generate_new_question(qtuple, logic_llm)
            write_output_to_file(
                generate_new_q_output, "./regenerate_question_generations", run_id
            )
            vet_question_loop(
                qtuple, total_retries, run_id=run_id.split("--subquestion--")[0]
            )  # going to get one hell of a call stack by the end of this, but it should be fine
    except Exception as e:
        print("!!ERROR!!")
        print(e)
        pass

    return (None, None, None, qtuple[3])


def vet_answer_relevance_loop(qa_tuple, total_retries, run_id):
    try:
        qtuple = qa_tuple
        print(
            f"\n\nStarting RELEVANCE loop for question: {qtuple[0]}, context: {qtuple[2]}"
        )
        passed_checks = 0
        times_checked = 0
        dissenting_reasoning = ""
        while times_checked < DOUBLE_CHECK_COUNTER:
            print(
                f"\n\nRELEVANCE CALL CHECK ANSWER: {qtuple[0]}, context: {qtuple[2]}, retries: {total_retries}, dissenting reasoning: {dissenting_reasoning}"
            )
            judgement, answer_relevancy_output = check_answer_relevancy_with_text(
                qtuple, logic_llm
            )
            write_output_to_file(
                answer_relevancy_output, "./check_answer_relevancy_generations", run_id
            )
            if not judgement[0]:  # if not relevant
                dissenting_reasoning = judgement[1]
            else:
                passed_checks += 1
            times_checked += 1
            if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                break
            failed_checks = times_checked - passed_checks
            if failed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                break

        if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
            print(f"\n\nRELEVANCE CHECKS PASSED")
            return vet_answer_accuracy_loop(qtuple, total_retries, run_id)
        else:
            print(f"\n\nRELEVANCE CHECKS FAILED - SENDING BACK TO QUESTION LOOP")
            total_retries += 1
            qtuple, generate_new_q_output = generate_new_question(qtuple, logic_llm)
            write_output_to_file(
                generate_new_q_output, "./regenerate_question_generations", run_id
            )
            return vet_question_loop(
                qtuple, total_retries, run_id=run_id.split("--subquestion--")[0]
            )
    except Exception as e:
        print("!!ERROR!!")
        print(e)
        pass

    return (None, None, None, qtuple[3])


def vet_question_loop(qa_tuple, question_group_id=None, total_retries=0):
    try:
        qtuple = qa_tuple
        print(
            f"\n\nStarting QUESTION loop for question: {qtuple[0]}, context: {qtuple[2]}"
        )
        while total_retries <= 4:
            run_id = question_group_id + "--subquestion--" + make_id()
            passed_checks = 0
            times_checked = 0
            dissenting_reasoning = ""
            while times_checked < DOUBLE_CHECK_COUNTER:
                print(
                    f"\n\nQUESTION CALL CHECK ANSWER: {qtuple[0]}, context: {qtuple[2]}, retries: {total_retries}, dissenting reasoning: {dissenting_reasoning}"
                )
                judgement, check_q_output = check_question(qtuple, logic_llm)
                write_output_to_file(
                    check_q_output, "./check_question_generations", run_id
                )
                if not judgement[0]:  # if not relevant
                    dissenting_reasoning = judgement[1]
                else:
                    passed_checks += 1
                times_checked += 1
                if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                    break
                failed_checks = times_checked - passed_checks
                if failed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                    break

            if passed_checks >= ceil(
                DOUBLE_CHECK_COUNTER / 2
            ):  # if all question checks passed
                print(f"\n\nQUESTION CHECKS PASSED retries: {total_retries}")
                return vet_answer_relevance_loop(qtuple, total_retries, run_id)
            else:
                # Generate new question and restart the loop
                print(
                    f"\n\nQUESTION CHECKS FAILED - GENERATING NEW QUESTION retries: {total_retries}"
                )
                total_retries += 1
                if (
                    total_retries <= 4
                ):  # only regen question if we're not already at max regens
                    qtuple, generate_new_q_output = generate_new_question(
                        qtuple, logic_llm
                    )
                    write_output_to_file(
                        generate_new_q_output,
                        "./regenerate_question_generations",
                        run_id,
                    )
                    print("New question: ", qtuple)
                # no calling of vet_question_loop, since we're already in a while loop
    except Exception as e:
        print("!!ERROR!!")
        print(e)

    return (None, None, None, qtuple[3])

In [ ]:
# control flow
import json
import os
from tqdm import tqdm
import glob

# Directory for QA tuples
qa_tuples_dir = "./qatuples_raw"
if not os.path.exists(qa_tuples_dir):
    os.makedirs(qa_tuples_dir)

# Initialize vetted_qa_tuples
vetted_qa_tuples = []  # tuple list of qa tuples that have been judged good

# Attempt to initialize filtered_worthy_for_questions
try:
    _ = filtered_worthy_for_questions
except NameError:
    filtered_worthy_for_questions = []

if not filtered_worthy_for_questions:
    # Load all files in the qa_tuples_dir if filtered_worthy_for_questions is not initialized
    existing_files = glob.glob(os.path.join(qa_tuples_dir, "*.json"))
    for file_path in existing_files:
        with open(file_path, "r") as file:
            qa_tuple = tuple(json.load(file))
        vetted_qa_tuples.append(qa_tuple)


else:
    for idx, para in enumerate(tqdm(filtered_worthy_for_questions)):
        # for idx, para in enumerate(tqdm(filtered_worthy_for_questions[:10])): # Use this instead if you are just testing all steps of the notebook
        try:
            existing_files = glob.glob(
                os.path.join(qa_tuples_dir, f"para_{idx}_*.json")
            )  # check if qs already exist

            if len(existing_files) > 0:  # If files exist, skip this paragraph entirely
                print(f"Skipping para_{idx} as files already exist; loading said files")
                for file_path in existing_files:
                    with open(file_path, "r") as file:
                        qa_tuple = tuple(json.load(file))
                    vetted_qa_tuples.append(qa_tuple)
                continue

            question_group_id = make_id()
            print(f"\n\n\nOUTER LOOP CALL GENERATE QPLAN para: {para}, \n\n idx: {idx}")
            plan, questions_plan_output = generate_questions_plan(para, logic_llm)
            write_output_to_file(
                questions_plan_output, "./question_plan_generations", question_group_id
            )
            print(
                f"\n\n\nOUTER LOOP CALL GENERATE Q: {para}, \n\n idx: {idx} \n\n plan: {plan}"
            )
            question_answer_tuples, question_generation_output = generate_questions(
                para, plan, logic_llm
            )
            write_output_to_file(
                question_generation_output,
                "./question_generation_generations",
                question_group_id,
            )
            for qnum, question_answer_tuple in enumerate(question_answer_tuples):
                print(
                    f"\n\n=======!!=BEGIN VETTING QA TUPLE {idx}_{qnum}=!!=======\n\n"
                )
                good_qa_tuple = vet_question_loop(
                    question_answer_tuple, question_group_id=question_group_id
                )

                # Write resulting question file if the tuple is not None
                if good_qa_tuple[0] is not None:
                    file_path = os.path.join(qa_tuples_dir, f"para_{idx}_q_{qnum}.json")
                    with open(file_path, "w") as file:
                        json.dump(good_qa_tuple, file, indent=4)

                vetted_qa_tuples.append(
                    good_qa_tuple
                )  # We must filter out all None values at the end; but appending Nones lets us know where things went wrong, and how often.
        except Exception as e:
            print(f"Q ERROR: {e}")

In [ ]:
print(
    "-------------- QUESTIONS CREATED ------------- STATS SO FAR (may be wrong if run was continued from interruption):"
)
nones = list(filter(lambda x: x[0] is None, vetted_qa_tuples))
print(f"Nones: {len(nones)}")
print(f"Non-nones: {len(vetted_qa_tuples) - len(nones)}")
print(f"Total: {len(vetted_qa_tuples)}")
# filter out all None values
vetted_qa_tuples = [qa for qa in vetted_qa_tuples if qa[0] is not None]
print("---------------- ONTO EXAMPLES GENERATION-------------------")

In [ ]:
# Check for and fix the common mistake: mentioning "the text".
# TODO refactor to be continuable, should take like 30 mins at most

writepath = "./qatuples_revised"


# Assuming vetted_qa_tuples is a list that might or might not exist
try:
    _ = vetted_qa_tuples
except NameError:
    vetted_qa_tuples = []


# Load all files at the start if vetted_qa_tuples is empty
if not vetted_qa_tuples:
    # Check if the directory exists
    if os.path.exists(writepath):
        # List all files in directory
        for file_name in os.listdir(writepath):
            file_path = os.path.join(writepath, file_name)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    print(f"Loading file: {file_path}")
                    if content == "failed":
                        vetted_qa_tuples.append(None)
                    else:
                        try:
                            data = json.loads(content)
                            vetted_qa_tuples.append(
                                (data[0], data[1], data[2], data[3])
                            )
                        except json.JSONDecodeError:
                            print("JSON decode error with the contents:", content)
                            vetted_qa_tuples.append(None)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

else:
    old_tuples = vetted_qa_tuples.copy()

    for idx, tup in enumerate(vetted_qa_tuples):
        file_path = os.path.join(writepath, f"revised_{idx}.json")
        if os.path.exists(file_path):
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()  # Read the file once and store its content
                print(file_path)
                if content == "failed":
                    print("Loaded failed file")
                    vetted_qa_tuples[idx] = None
                    continue
                print("Loaded file:")
                print(content)
                # Reset the file pointer to the beginning if you need to read again or convert string back to JSON
                try:
                    data = json.loads(content)  # Convert the string back to JSON
                    vetted_qa_tuples[idx] = (data[0], data[1], data[2], data[3])
                    continue
                except json.JSONDecodeError:
                    print("JSON decode error with the contents:", content)
                    # Handle the error appropriately

        try:
            revision_id = make_id()
            revision, revision_output = check_qatuple_context(tup, logic_llm)
            write_output_to_file(
                revision_output, "./question_context_revision_generations", revision_id
            )  # incidentally, identifying the problem and fixing it in the same step (without another planning step) works a lot better than identifying it and then trying to fix it in the next step.
            if isinstance(revision[0], str):  # if the thing was reworded
                vetted_qa_tuples[idx] = revision
            elif not revision[0]:
                vetted_qa_tuples[
                    idx
                ] = None  # prepare item for deletion later; right now we just store it as None because indexes
            # else, if it passed, we just leave it be.

            # Write in-progress
            if not os.path.exists(writepath):
                os.makedirs(writepath)

            if vetted_qa_tuples[idx]:
                with open(file_path, "w") as file:
                    json.dump(vetted_qa_tuples[idx], file, indent=4)
            else:
                with open(file_path, "w") as file:
                    file.write("failed")

        except Exception as e:
            print("!!! ERROR!", e)

In [ ]:
# Print stats related to revised qatuples, and filter out nones (questions that were unanswerable due to lack of context).
import json
import os

print("-------------- QUESTIONS REVISED ------------- STATS SO FAR:")
nones = list(filter(lambda x: x is None, vetted_qa_tuples))
print(f"Nones: {len(nones)}")
print(f"Non-nones: {len(vetted_qa_tuples) - len(nones)}")
print(f"Total: {len(vetted_qa_tuples)}")
# filter out all None values
vetted_qa_tuples = [qa for qa in vetted_qa_tuples if qa is not None]
print("---------------- ONTO EXAMPLES GENERATION-------------------")

In [ ]:
# Group tuples for multiturn example generation (by chunk of source text) and then run that helper (so that we can make multiturn conversations from questions based on the same paragraphs)
def group_by_text(tuples_list):
    # Dictionary to hold the groups with text as the key
    groups = {}

    # Iterate over each tuple in the list
    for question, answer, text, textname in tuples_list:
        # If the text is not yet a key in the dictionary, add it with an empty list
        if text not in groups:
            groups[text] = []

        # Append the current tuple to the appropriate list
        groups[text].append((question, answer, text, textname))

    # Return the values of the dictionary, which are the lists of tuples grouped by text; also remove duplicates
    return [identify_duplicates(group) for group in list(groups.values())]

In [ ]:
qa_tuples_by_paragraph = group_by_text(vetted_qa_tuples)

In [ ]:
from math import ceil

# multiturn helpers
# These will probably be used for multiturn rapid-fire answering.


# Idea: use multiple short answers to train the task of answering multiple questions in one response. Two-three short answers per response should be enough.
def make_multiturn_character(qa_tuples, conv_id):
    if (
        ASSISTANT_MODE
    ):  # If assistant mode is on, multiturn convs will have hardcoded information in its prompt file; but we still need to put something in the file
        return "will_be_replaced", "will_be_replaced"
    plan, instructions, card_plan_output = create_character_card_plan_many_tuples(
        qa_tuples, logic_llm
    )  # I will reuse the many tuples function for short question-answers, there's a lot of prompting in here already
    write_output_to_file(card_plan_output, "./multiturn_card_plan_generations", conv_id)
    char, char_output = create_character_card_many_tuples(
        qa_tuples, plan, instructions, logic_llm
    )  # creates a character card
    write_output_to_file(char_output, "./multiturn_card_generations", conv_id)
    return char, instructions


def make_multiturn_scenario(qa_tuples, character, conv_id):
    if (
        ASSISTANT_MODE
    ):  # If assistant mode is on, multiturn convs will have hardcoded information in its prompt file; but we still need to put something in the file
        return "will_be_replaced", "will_be_replaced"
    plan, scenario_plan_output = create_scenario_plan_many_tuples(
        qa_tuples, character, logic_llm
    )
    write_output_to_file(
        scenario_plan_output, "./multiturn_scenario_plan_generations", conv_id
    )
    scenario, scenario_output = create_scenario_many_tuples(
        qa_tuples, character, plan, logic_llm
    )  # creates a scenario based on a character card and question/answer tuple
    write_output_to_file(scenario_output, "./multiturn_scenario_generations", conv_id)
    return scenario, plan


def make_multiturn_conversation_info(qa_tuples, logic_llm):
    conv_id = make_id()
    if (
        ASSISTANT_MODE
    ):  # If assistant mode is on, multiturn convs will have hardcoded information in its prompt file; but we still need to put something in the file
        return (qa_tuples, "will", "be", "replaced", conv_id)
    # thought_plan = create_thought_plan_many_tuples(qa_tuples,character,scenario,logic_llm) # There IS a way to make multiturn chain of thought answering work: generate each pair of messages using a separate prompt or a separate function, each of which has only the thought plan for that question/answer pair. But simply cramming in all the step-by-step things will confuse the hell out of the poor model. So for the first release version we're skipping it and just giving the response, with no reasoning, in the multiturn convs.
    character, instructions = make_multiturn_character(qa_tuples, conv_id)
    scenario, scenario_plan = make_multiturn_scenario(qa_tuples, character, conv_id)

    return (qa_tuples, character, scenario, scenario_plan, conv_id)

In [ ]:
import os
import json
import random
import itertools


if not os.path.exists(multi_turn_convs_info_dir):
    os.makedirs(multi_turn_convs_info_dir)

In [ ]:
multi_turn_convs_info = []
for idx, group in enumerate(qa_tuples_by_paragraph):
    all_permutations = list(itertools.permutations(group))

    sample_size = min(REARRANGEMENTS_TO_TAKE, len(all_permutations))
    sampled_permutations = random.sample(all_permutations, sample_size)

    group_convs_info = []

    for iter, perm in enumerate(sampled_permutations):
        file_path = os.path.join(multi_turn_convs_info_dir, f"info_{idx}_{iter}.json")

        # Skip if file already exists
        if not os.path.exists(file_path):
            info = make_multiturn_conversation_info(perm, logic_llm)

            if info is not None:
                with open(file_path, "w") as file:
                    json.dump(info, file, indent=4)

            group_convs_info.append(info)
        else:
            print(f"Skipped generating {file_path} as it already exists")

    multi_turn_convs_info.append(group_convs_info)

In [ ]:
del logic_llm  # Apparently frees the model, haven't tested this extensively

# Stop Here, Restart the Notebook, and Reimport Everything IF you are doing 2-step Generation (where you do the easy bits with a small model, and the hard bit with a large one).

In [ ]:
# Load model into vram
from llama_cpp import Llama

logic_llm = Llama(
    model_path=LARGE_LOGICAL_MODEL,
    n_gqa=8,
    offload_kqv=True,
    n_ctx=12000,
    rope_freq_scale=0.33,
    n_gpu_layers=100,
    verbose=False,
)  # load the logical LLM and offload everything

In [ ]:
import os
import json


def read_json_files_info(directory):
    # Create a list to hold the tuples
    tuple_list = []

    # Get all the .json files in the directory, sorted
    json_files = sorted([f for f in os.listdir(directory) if f.endswith(".json")])

    # Read each file and convert the contents
    for file in json_files:
        with open(os.path.join(directory, file), "r") as f:
            data = json.load(f)
            # Ensure the data is in the correct format before converting to tuple
            if (
                isinstance(data, list)
                and len(data) == 5
                and isinstance(data[0], list)
                and all(len(item) == 4 for item in data[0])
                and all(isinstance(i, str) for i in data[1:])
            ):
                tuple_list.append((data[0], data[1], data[2], data[3], data[4]))

    return tuple_list


convs_info = read_json_files_info(multi_turn_convs_info_dir)

In [ ]:
import os
import json
import random
import itertools

multi_turn_convs_dir = "./multi_turn_convs"
if not os.path.exists(multi_turn_convs_dir):
    os.makedirs(multi_turn_convs_dir)

multi_turn_convs = []
for idx, info in enumerate(convs_info):
    file_path = os.path.join(multi_turn_convs_dir, f"conv_{idx}.json")

    # Skip if file already exists
    if not os.path.exists(file_path):
        conv = make_multiturn_conversation(info, logic_llm)
        final_conv = ensure_multiple_answers_are_same(info, conv, logic_llm)

        if final_conv is not None:
            with open(file_path, "w") as file:
                json.dump(final_conv, file, indent=4)

        multi_turn_convs.append(final_conv)
    else:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            multi_turn_convs.append(data)
        print(f"Skipped generating {file_path} as it already exists")

# Yay! Now you have a dataset!
### GPT wrote the cell below. I think it successfully converts things to ShareGPT format for use with axolotl, but I am not sure because I don't know that format very well and haven't used Axolotl. However, the json produced by the second function looks fine.

In [ ]:
import os
import json


def convert_directory_to_list(directory_path):
    master_list = []
    simplified_list = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            filepath = os.path.join(directory_path, filename)
            with open(filepath, "r") as file:
                data = json.load(file)
                if isinstance(data, list) and all(
                    isinstance(item, (list, str)) for item in data
                ):
                    master_list.append(data)

                    # Extract and process conversation
                    conversation, primary_char_desc = data[0], data[1]
                    primary_char_name = extract_name(primary_char_desc)
                    dialogues = extract_conversation(conversation)

                    # Convert to simplified format
                    simplified_conversations = []
                    for i, (charname, message) in enumerate(
                        dialogues
                    ):  # Skipping the first message
                        from_person = (
                            "human" if charname == primary_char_name else "gpt"
                        )
                        simplified_conversations.append(
                            {"from": from_person, "value": f"{charname}: {message}"}
                        )

                    if simplified_conversations:  # If there are any conversations
                        simplified_list.append(
                            {"conversations": simplified_conversations}
                        )

    # Write the master list to a new .jsonl file
    with open("master_list.jsonl", "w") as file:
        for item in master_list:
            file.write(json.dumps(item) + "\n")

    # Write the simplified data to a different .jsonl file
    with open("simplified_data.jsonl", "w") as file:
        for item in simplified_list:
            file.write(json.dumps(item) + "\n")

    print(
        "Conversion complete. Master list written to 'master_list.json'. Simplified data written to 'simplified_data.json'."
    )


convert_directory_to_list("./multi_turn_convs/")

# Write version where each line in the conversation is separate, so that you do not need to write an extract_convs function if you want a custom format.


def convert_directory_and_process_conversations(directory_path):
    master_list = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            filepath = os.path.join(directory_path, filename)
            with open(filepath, "r") as file:
                data = json.load(file)

                if isinstance(data, list) and all(
                    isinstance(item, (list, str)) for item in data
                ):
                    # Extract and process the conversation part
                    conversations = extract_conversation(data[0])
                    # Convert tuples back to the formatted string as required
                    data[0] = [
                        f"{charname}: {message}" for charname, message in conversations
                    ]
                    master_list.append(data)
                else:
                    print(f"File {filename} is not in the expected format.")

    # Write the master list to a new file
    with open("processed_master_list.json", "w") as file:
        json.dump(master_list, file)

    print(
        "Conversion complete. The processed master list is written to 'processed_master_list.json'."
    )


# Usage: replace 'path_to_directory' with the actual directory path
convert_directory_and_process_conversations("./multi_turn_convs/")

In [ ]:
with open("./processed_master_list.json") as f:
    first = f.read()
    data = json.loads(first)


# For curiosity's sake, you can find out how many lines of dialogue you generated
def filter_and_flatten(lst):
    # Initialize an empty list to hold the flattened elements
    flat_list = []

    # Loop through each sublist in the main list
    for sublst in lst:
        # Check if the first element of the sublist is itself a list (subsublist1)
        if isinstance(sublst[0], list):
            # Extend the flat_list with the elements from subsublist1
            flat_list.extend(sublst[0])

    return flat_list


len(filter_and_flatten(data))